# Preparation

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to check out the [SAP generative AI hub SDK project description](https://pypi.org/project/generative-ai-hub-sdk/) for more details. Below you will find an example for authenticating via environment variables using this very notebook.

> **WARNING:**
> Below code should never be used in production scenarios and is only for the purpose of illustrating which environment variables to use!
> Credentials should never be defined in code!

In [ ]:
import os
import json

with open('creds.json') as f:
    creds = json.load(f)
    
os.environ["AICORE_AUTH_URL"] = f"{creds['url']}/oauth/token"
os.environ["AICORE_BASE_URL"] = creds['serviceurls']['AI_API_URL']
os.environ["AICORE_CLIENT_ID"] = creds['clientid']
os.environ["AICORE_CLIENT_SECRET"] = creds['clientsecret']
os.environ["AICORE_ORCHESTRATION_DEPLOYMENT_URL"] = "https://aicore-proxy-ai180p104road.cfapps.eu12.hana.ondemand.com/v2/inference/deployments/d809af1946272325"
os.environ["AICORE_RESOURCE_GROUP"] = "default"

You will need to authenticate in every exercise.

You must also create deployments for the models you want to consume via AI Launchpad or API.

# Usage

### Sending a request to an embedding model.

In [3]:
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

embedding_model = OpenAIEmbeddings(proxy_model_name='text-embedding-3-small')
response = embedding_model.embed_query('Every decoding is another encoding.')
print(response)

[0.02729123644530773, -0.010414490476250648, -0.016543924808502197, -0.009471501223742962, 0.02547459304332733, 0.02572420798242092, 0.09135907888412476, -0.01693221554160118, 0.006895613390952349, -0.041602492332458496, 0.058742720633745193, -0.014311257749795914, -0.04534671828150749, -0.01987212523818016, 0.04346073791384697, -0.03808014839887619, 0.013964570127427578, 0.034086309373378754, 0.0036644863430410624, 0.050144873559474945, 0.006815874949097633, -0.04850850626826286, 0.06933748722076416, 0.032865971326828, 0.0305362306535244, -0.02797074243426323, -0.004552006255835295, 0.07743610441684723, 0.04096458852291107, -0.03461327403783798, -0.010275815613567829, -0.028927600011229515, 0.01608629710972309, 0.01815255545079708, 0.045263513922691345, 0.012210331857204437, -0.00738444272428751, -0.015212644822895527, -0.012584754265844822, 0.02478121779859066, 0.03514023870229721, -0.04371035471558571, -0.049451496452093124, -0.009339760057628155, 0.011454553343355656, 0.02761018835

### Sending request to an Anthropic model.

In [4]:
from gen_ai_hub.proxy.native.amazon.clients import Session

model = Session().client(model_name='anthropic--claude-3-sonnet')
messages = [{"role": "user", "content": [{'text': "I'm applying to be a data scientist at SAP. What should I include in my resume?"}]}]
response = model.converse(messages=messages,
    inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
)
print(response['output']['message']['content'][0]['text'])

When applying for a data scientist role at SAP, you should tailor your resume to highlight your relevant skills, experience, and achievements in data analysis, machine learning, and statistical modeling. Here are some key elements to consider including:

1. Education: List your academic qualifications, including your degree(s) in a relevant field such as computer science, statistics, mathematics, or a related quantitative discipline.

2. Technical Skills: Highlight your proficiency in programming languages commonly used in data science, such as Python, R, SQL, and any relevant libraries or frameworks (e.g., NumPy, Pandas, Scikit-learn, TensorFlow, Keras). Also, mention your expertise in data manipulation, visualization, and machine learning techniques.

3. Data Science Projects: Provide details about your relevant data science projects, internships, or research work. Describe the problem you solved, the methodologies you used, and the impact or results achieved. Quantify your achieveme

### Sending request to a GPT model.

Non-Streaming:

In [5]:
from gen_ai_hub.proxy.native.openai import chat

messages = [
            {
              "role": "user",
              "content": "Write me a long Poem about SAP?"
            }
        ]
kwargs = { "max_tokens": 500, 'stream': False}
ns_response = chat.completions.create(**dict(model_name='gpt-4o-mini', messages=messages, **kwargs))
print(ns_response.choices)

[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="### Ode to SAP: The Heartbeat of Enterprise\n\nIn the realm where business dreams align,  \nA sturdy framework, where ideas entwine,  \nStands SAP, a beacon, a guiding star,  \nA tapestry woven, both near and far.  \n\nFrom humble beginnings in the sixties' dawn,  \nWhere visions of progress and tech were drawn,  \nIt flourished like blossoms, in spring's gentle air,  \nA software solution beyond compare.  \n\nWith modules designed to streamline the flow,  \nFinance and logistics, where insights bestow,  \nIn the vast world of data, it carves out a path,  \nForging connections, igniting the math.  \n\nOh, ERP wizard, you're more than a tool,  \nYou empower the shepherds, the dreamers, the school,  \nFrom sales to procurement, each function aligned,  \nAn orchestra playing, with harmony twined.  \n\nIn warehouses bustling, the inventory flows,  \nWith real-time insights, production just glows,

Streaming:

In [25]:
from gen_ai_hub.proxy.native.openai import chat

messages = [
            {
              "role": "user",
              "content": "Write me a long Poem about SAP?"
            }
        ]
kwargs = { "max_tokens": 500, "stream": True}
s_response = chat.completions.create(**dict(model_name='gpt-4o-mini', messages=messages, **kwargs))
for chunk in s_response:
    print(chunk)
    print(chunk.choices)
    print("****************")

ChatCompletionChunk(id='', choices=[], created=0, model='', object='', service_tier=None, system_fingerprint=None, usage=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
[]
****************
ChatCompletionChunk(id='chatcmpl-ALAQheTYi6ylyVPqsbQlMspkZ7fKL', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results={})], created=1729608343, model='gpt-4o-mini', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_d54531d9eb', usage=None)
[Choice(delta=ChoiceDelta(content=None, function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results=

In [6]:
print(ns_response.choices[0].message.content)

**In the Realm of SAP**

In the heart of enterprises, bustling and grand,  
Lies a system so mighty, with a guiding hand.  
A symphony of data, a dance in the cloud,  
With SAP leading, so steadfast and proud.  

From the hum of the factory, where machines sing,  
To the boardroom’s whispers of strategy’s swing,  
SAP’s essence weaves through each passing day,  
Transforming the numbers in intricate ways.  

A tapestry woven, where processes flow,  
From procurement’s cradle, where budgets can grow,  
To sales that ignite with a vibrant appeal,  
SAP nurtures the rhythm, a symphonic reel.  

With modules aplenty, each crafted with care,  
From Finance to HR, their power laid bare.  
A canvas of functions, a toolkit divine,  
Where the art of decision finds rhythm and rhyme.  

In S/4HANA’s embrace, the future’s mindset,  
The analytics pulse, like a star brightly set.  
With insights unveiled, and data in sight,  
Businesses flourish, evolving their flight.  

The storm of the market m

### Sending request to other models.

In [2]:
from gen_ai_hub.proxy.native.openai import chat
from gen_ai_hub.proxy.native.google_vertexai.clients import GenerativeModel
from gen_ai_hub.proxy.native.amazon.clients import Session
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

LLMS = {'amazon': ['amazon--titan-text-express', 'amazon--titan-text-lite'], 
        'anthropic': ['anthropic--claude-3-haiku', 'anthropic--claude-3-opus', 'anthropic--claude-3-sonnet', 'anthropic--claude-3.5-sonnet'],
        'openai': ['gpt-4o-mini', 'gpt-4o', 'gpt-4'],
        'google': ['gemini-1.5-flash', 'gemini-1.5-pro', 'gemini-1.0-pro'],
        'meta': ['meta--llama3.1-70b-instruct']}

def get_model_response(model, system_instruction, prompt):
    if 'openai' in LLMS and model in LLMS['openai']:  
        messages = [{"role": "system", "content": system_instruction}, {"role": "user", "content": prompt}]
        return chat.completions.create(**dict(model_name=model, messages=messages)).choices[0].message.content
    elif 'meta' in LLMS and model in LLMS['meta']:
        messages = [{"role": "system", "content": system_instruction}, {"role": "user", "content": prompt}]
        return chat.completions.create(**dict(model_name=model, messages=messages)).choices[0].message.content
    elif 'google' in LLMS and model in LLMS['google']:
        proxy_client = get_proxy_client('gen-ai-hub')
        kwargs = dict({'model_name': model, 'system_instruction':[system_instruction], 'generation_config': {"response_mime_type": "application/json"}})
        model = GenerativeModel(proxy_client=proxy_client, **kwargs)
        model_response = model.generate_content([prompt])
        return model_response.candidates[0].content.parts[0].text
    elif 'amazon' in LLMS and model in LLMS['amazon']:
        model = Session().client(model_name=model)
        body = json.dumps(
            {
                    "inputText":f"{system_instruction}\n{prompt}",
                    "textGenerationConfig": {
                        "maxTokenCount": 3072,
                        "stopSequences": [],
                        "temperature": 0.7,
                        "topP": 0.9,
                    },
            }
        )
        response = model.invoke_model(body=body)
        response_body = json.loads(response.get("body").read())
        return response_body['results'][0]['outputText']
    elif 'anthropic' in LLMS and model in LLMS['anthropic']:
        model = Session().client(model_name=model)
        messages = [{"role": "user", "content": [{'text': f'{system_instruction}\n{prompt}'}]}]
        response = model.converse(messages=messages,
            inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
        )
        return response['output']['message']['content'][0]['text']

In [ ]:
print(get_model_response('meta--llama3.1-70b-instruct', 'You are comedian, and you must talk with humor.', 'Tell me a really mean joke.'))

# Summary

Within this exercise you learned how to interact with LLMs and embedding models directly. Also, you played around with LLM parameters like temperature. Let's explore the orchestration module in the following exercises. Continue to [Exercise 2 - Orchestration Templating](./ex2.ipynb).